# 2 Aufteilen und Säubern der Daten und Erfassen von Metadaten
## Der Code für Kapitel 3, 4 und 5 in https://dennisfriedl.de/blog/text_mining_amtspresse
### Die Pfade müssen angepasst werden!

In [10]:
# Imports von Modulen
import os # Für Dateien
import re # Regex
import csv # CSV

In [11]:
# Umzustellendene Variablen
paper = "ProvinzialCorrespondenz" # umstellen auf NeuesteMittheilungen oder ProvinzialCorrespondenz

In [12]:
#### FUNCTIONS ####
def xmlDeleter(l): # Löschung der XML-Tags
    for i in range(len(l)): 
        l[i] = re.sub("<br/>", " ", l[i]) # Statt <br/> ein Leerzeichen, da sonst Wörter zusammenrücken würden
        l[i] = re.sub("<\/?[a-z]*\/?>", "", l[i]) # Regex für XML-Tags. Wird durch nichts ersetzt.

def stripper(l): # Befreit die Artikel von Leerzeichen und Umbrüchen am Angang und am Ende
    for i in range(len(l)):
        l[i] = l[i].replace("\n", " ")
        l[i] = l[i].strip()

In [13]:
for file in os.listdir("../Data/TXT/{}".format(paper)): # Löscht am Anfang alle alten Daten
    os.remove("../Data/TXT/{}/{}".format(paper, file))

In [14]:
# Definition der gewünschten Metadaten:
csvRow = {"Dateiname": None, "Zeitungsname": None, "Tag": None, "Monat": None, "Jahr": None, "Jahr_Monat": None ,"Jahr_Monat_Tag": None, "Artikelanzahl_in_der_Ausgabe": None, "Stelle_in_der_Artikelanzahl": None, "Ueberschrift": None, "Woerteranzahl_Artikel": None, "Woerteranzahl_Ausgabe": None}
csvRow["Zeitungsname"] = paper;

In [15]:
# Ertsellen der CSV-Datei und schreiben des Headers
with open("../Data/CSV/{}/metadata.csv".format(paper), mode='w', newline='', encoding="utf-8") as csvFile:
    fieldnames = list(csvRow.keys())
    writer = csv.DictWriter(csvFile, fieldnames=fieldnames)
    writer.writeheader()

In [16]:
for file in os.listdir("../Data/XML/{}".format(paper)): # Geht jede Datei im Ordner durch
    
    csvContent = list() # wird jede Ausgabe zurückgesetzt
    # die gesamte Ausgabe gleich:
    csvRow["Jahr"] = file[:4]
    csvRow["Monat"] = file[4:6]
    csvRow["Tag"] = file[6:8]
    csvRow["Jahr_Monat"] = csvRow["Jahr"] + "-" + csvRow["Monat"]
    csvRow["Jahr_Monat_Tag"] = csvRow["Jahr"] + "-" + csvRow["Monat"] + "-" + csvRow["Tag"]
    
    with open("../Data/XML/{}/{}".format(paper, file),"r", encoding="utf-8") as xmlFile: # Öffnet die Datei
        data = xmlFile.read()

    data = data.split("<fusszeile>")[0] # Trennt die Fußzeile ab
    data = data.split("<untertitel>")[1:] # Teilt nach Untertiteln und trennt die Kopfzeile ab
    
    for i in range(len(data)): # Trennt bei mehrmaligem Auftreten von "<hr/>" die Artikelsammlung nochmal in Artikel anhand von "<hr/>"
        if data[i].count("<hr/>") > 1:
            data[i] = data[i].split("<hr/>")
    
    #hängt die durch die <hr/>-Trennung entstandenen Artikel an die Liste an
    oldData = data
    data = list()
    for i in oldData:
        if type(i) is str:
            data.append(i)
        elif type(i) is list:
            for j in i:
                if len(j) > 1:
                    data.append(j)
                    
    stripper(data)

    ueberueberschriften = [x for x in data if x.endswith("</untertitel>")] # extrahiert Überüberschriften
    
    data = [x for x in data if not x.endswith("</untertitel>")] # Löscht die Überüberschriften aus der Artikelliste
    
    # Überschriften speichern für die Metadaten
    ueberschriften = list(range(len(data)))
    for i in range(len(data)):
        if "</untertitel>" in data[i]:
            ueberschriften[i], data[i] = data[i].split("</untertitel>")
    for i in range(len(ueberschriften)):
        if type(ueberschriften[i]) == int:
            ueberschriften[i] = ueberschriften[i-1]
    
    xmlDeleter(data)
    xmlDeleter(ueberschriften)
    xmlDeleter(ueberueberschriften)
    stripper(data)
    stripper(ueberschriften)
    stripper(ueberueberschriften)
    
    # Die Artikel in Überüberschriften werden oft einfach römisch durchnummeriert
    # Hier soll verhindert werden, dass diese römischen Zahlen Teil der Überschriften in den Metadaten werden
    deleteReady = False
    for i in range(len(ueberschriften)):
        if ueberueberschriften and ueberschriften[i] in ["I.", "II.", "III.","IV.","V.", "VI.", "VII.","VIII.""VIII.","IX.","X.","XI.","XII.","XIII.","XIV.","XV.",]:
            ueberschriften[i] = ueberueberschriften[0]
            deleteReady = True
        elif deleteReady:
            ueberueberschriften.pop(0)
            deleteReady = False
    
    data = [x for x in data if len(x.split(" ")) > 3] # Behält nur Artikel mit mehr als drei Wörtern
    
    # Speichern weiterer Metadaten
    csvRow["Artikelanzahl_in_der_Ausgabe"] = len(data)
    woerterAusgabe = 0
    for i in range(len(data)):
        woerterAusgabe += len(data[i].split(" "))
    csvRow["Woerteranzahl_Ausgabe"] = woerterAusgabe
    
    for i in range(len(data)): 
        csvRow["Dateiname"] = str(file[:-4]) + "-" + str(i+1) + ".txt"
        csvRow["Stelle_in_der_Artikelanzahl"] = i+1
        csvRow["Ueberschrift"] = ueberschriften[i]
        csvRow["Woerteranzahl_Artikel"] = len(data[i].split(" "))
        woerterAusgabe += len(data[i].split(" "))
        csvContent.append(csvRow.copy())
    
    for i in range(len(data)): # Speichert die Artikel als einzelne Txt-Dateien
        with open("../Data/TXT/{}/{}-".format(paper, file[:-4]) + str(i+1) + ".txt","w+", encoding="utf-8") as txtFile:
            txtFile.write(data[i])
   
    # Anhängen der Metadaten an die CSV-Datei
    with open("../Data/CSV/{}/metadata.csv".format(paper), mode='a', newline='', encoding="utf-8") as csvFile: 
        fieldnames = list(csvRow.keys())
        writer = csv.DictWriter(csvFile, fieldnames=fieldnames)
        for row in csvContent:
            writer.writerow(row)
            
    